### Importing Libraries and Constants Definition

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

DATA_PATH = '../data/public/'
PROPOSALS_FILE = DATA_PATH + 'brasilparticipativo.presidencia.gov.br-open-data-proposals.csv'
VOCAB_FILE = DATA_PATH + 'vocabulario-controlado-basico-vcb-lista-alfabetica.txt'
TOP_N_TOPICS = 5

### Functions Definition

In [2]:
def process_vocab_file(vocab_file):
    ROWS_TO_SKIP = 8
    vocab_dict = {}
    with open(vocab_file, 'r', encoding='utf-8') as file:
        current_term = ''
        for line in file.read().splitlines()[ROWS_TO_SKIP:]:
            if not line.startswith('\t') and line != '':
                current_term = line
                vocab_dict[current_term] = {
                    'USE': [], # Use
                    'DF': [],  # Definição
                    'UP': [],  # Termos não preferenciais
                    'TG': [],  # Termo geral
                    'TR': [],  # Termos relacionados
                    'TE': [],  # Termos específicos
                    'EQ': [],  # Equivalente
                    'Nota de escopo': [],
                }
            elif line != '':
                property = line.strip().split(':')
                key = property[0].strip()
                value = property[1].strip()
                vocab_dict[current_term][key].append(value)
    return vocab_dict

def process_vocab(vocab_file, exclusion_terms):
    vocab_json = process_vocab_file(vocab_file)
    vocab_list_processed = [key for key in vocab_json if key.lower() not in exclusion_terms]
    return vocab_list_processed

def load_and_preprocess_proposals(file_path):
    df = pd.read_csv(file_path, delimiter=';')
    
    # Remove rejected and withdrawn proposals
    df = df[~df['state'].isin(['rejected', 'withdrawn'])]
    
    # Replace <br> or <br/> or <br /> with spaces
    df['body/pt-BR'] = df['body/pt-BR'].str.replace(r'<br\s*/?>', ' ', regex=True)
    # Remove other HTML tags
    df['body/pt-BR'] = df['body/pt-BR'].str.replace(r'<[^>]*>', '', regex=True)
    # Remove text after "Orgão Responsável"
    df['body/pt-BR'] = df['body/pt-BR'].apply(lambda x: x.split('Órgão Responsável:')[0])
    
    relevant_cols = [
        'id',
        'category/id',
        'category/name/pt-BR',
        'title/pt-BR',
        'body/pt-BR',
        'supports',
        'followers',
        'comments',
        'published_at',
        'state', # REMOVE
        'url',
        'participatory_space/url',
    ]
    return df[relevant_cols]

def get_unique_themes(df):
    themes = df['category/name/pt-BR'].str.lower().unique().tolist()
    themes = [str(theme) for theme in themes if theme is not np.nan]
    themes = set([theme.split('- direito à')[-1].strip() for theme in themes])
    themes = set([theme.split('- direito ao')[-1].strip() for theme in themes])
    return themes


def classify_topics(sentences, topics, top_n=TOP_N_TOPICS):
    all_text = sentences + topics
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_text)

    classified_data = []
    for i, sentence in enumerate(sentences):
        cosine_similarities = cosine_similarity(tfidf_matrix[i:i+1], tfidf_matrix[len(sentences):]).flatten()
        top_n_indices = np.argsort(cosine_similarities)[-top_n:][::-1]
        top_n_topic_similarities = [(topics[idx], cosine_similarities[idx]) for idx in top_n_indices]
        
        # Append a tuple containing the sentence and its classified topics
        classified_data.append((sentence, top_n_topic_similarities))

    return classified_data

### Loading and preprocessing proposals

In [3]:
df_propostas = load_and_preprocess_proposals(PROPOSALS_FILE)
df_propostas.head(5)

,id,category/id,category/name/pt-BR,title/pt-BR,body/pt-BR,supports,followers,comments,published_at,state,url,participatory_space/url
0,1,30.0,Turismo,Turismo: esse é o Destino,Objetivo: Posicionar o turismo como vetor de d...,1,1,0,2023-05-10 10:03:41 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
1,8,31.0,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,1,0,0,2023-05-10 16:22:51 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
2,9,1.0,Agricultura e Pecuária,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,2,0,0,2023-05-10 16:35:47 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
3,10,27.0,Saúde,Atenção Primária à Saúde,"Fortalecer a Atenção Primária à Saúde, amplian...",20427,516,0,2023-05-10 16:42:43 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
4,11,27.0,Saúde,Atenção Especializada à Saúde,Ampliar o acesso às ações e serviços da Atençã...,18786,385,0,2023-05-10 16:41:01 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...


### Extracting sentences from proposals and titles

In [4]:
df_propostas['sentence'] = df_propostas['title/pt-BR'].str.lower() + ' ' + df_propostas['body/pt-BR'].str.lower()
sentences = df_propostas['sentence'].tolist()
sentences[:5]

['turismo: esse é o destino objetivo: posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.  ',
 'agricultura familiar e agroecologia objetivo: fortalecer a agricultura familiar em sua diversidade e a agroecologia, promovendo a inclusão socioeconômica, com fomento à produção sustentável e à geração de renda, contribuindo para a promoção da segurança alimentar e nutricional e da vida digna, com redução das desigualdades e mitigação das mudanças climáticas.  ',
 'agropecuária sustentável objetivo: contribuir para o desenvolvimento do setor agropecuário, com sustentabilidade ambiental, econômica e social, por meio do fomento e da intensificação da produção; da geração, compartilhamento e aplicação de conhecimento técnico-científico; e da regulamentação, certificação, auditoria e fiscalização agropecuária.',
 'atenção primária à saúde

### Getting unique themes

In [5]:
themes = get_unique_themes(df_propostas)
list(themes)[:5]

['relações institucionais',
 'sustentabilidade e ao meio ambiente',
 'agricultura e pecuária',
 'previdência social',
 'comunicações']

### Processing vocabulary

In [6]:
vocab_list = process_vocab(VOCAB_FILE, themes)
# Filter terms with more than 20 words
vocab_list = [term for term in vocab_list if len(term.split()) <= 20]
vocab_list[:10]

['2019-nCoV disease',
 '2FA',
 '2Factor Authentication',
 'AAE',
 'Ab-rogação',
 'Abacate',
 'Abacaxi',
 'Abalo sísmico',
 'Abalroamento aéreo',
 'Abalroamento de navios']

### Classifying topics

In [7]:
classified_topics = classify_topics(sentences, vocab_list)
classified_topics[:3]

[('turismo: esse é o destino objetivo: posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.  ',
  [('Turismo sustentável', 0.37312397508528844),
   ('Brasileiros', 0.33192226754144455),
   ('Turismo de saúde', 0.3023275661273092),
   ('Atividade turística', 0.29415037488258816),
   ('Turismo social', 0.290470541507436)]),
 ('agricultura familiar e agroecologia objetivo: fortalecer a agricultura familiar em sua diversidade e a agroecologia, promovendo a inclusão socioeconômica, com fomento à produção sustentável e à geração de renda, contribuindo para a promoção da segurança alimentar e nutricional e da vida digna, com redução das desigualdades e mitigação das mudanças climáticas.  ',
  [('Agricultura familiar', 0.41338616246804105),
   ('Agricultura sustentável', 0.3096735086895474),
   ('Agricultura, produção', 0.307187798020645

### Processing classified topics

In [8]:
classified_topics_processed = [[sentence] + [f"{x[0]} ({round(x[1]*100)}%)" for x in topics] for sentence, topics in classified_topics]
classified_topics_processed[:3]

[['turismo: esse é o destino objetivo: posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.  ',
  'Turismo sustentável (37%)',
  'Brasileiros (33%)',
  'Turismo de saúde (30%)',
  'Atividade turística (29%)',
  'Turismo social (29%)'],
 ['agricultura familiar e agroecologia objetivo: fortalecer a agricultura familiar em sua diversidade e a agroecologia, promovendo a inclusão socioeconômica, com fomento à produção sustentável e à geração de renda, contribuindo para a promoção da segurança alimentar e nutricional e da vida digna, com redução das desigualdades e mitigação das mudanças climáticas.  ',
  'Agricultura familiar (41%)',
  'Agricultura sustentável (31%)',
  'Agricultura, produção (31%)',
  'Vida familiar (30%)',
  'Renda familiar (30%)'],
 ['agropecuária sustentável objetivo: contribuir para o desenvolvimento do setor agr

### Creating DataFrame with classified topics

In [9]:
columns = ['sentence'] + [f"Tópico {i}" for i in range(1, TOP_N_TOPICS + 1)]
df_classified = pd.DataFrame(classified_topics_processed, columns=columns)
df_classified.head(5)

,sentence,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5
0,turismo: esse é o destino objetivo: posicionar...,Turismo sustentável (37%),Brasileiros (33%),Turismo de saúde (30%),Atividade turística (29%),Turismo social (29%)
1,agricultura familiar e agroecologia objetivo: ...,Agricultura familiar (41%),Agricultura sustentável (31%),"Agricultura, produção (31%)",Vida familiar (30%),Renda familiar (30%)
2,agropecuária sustentável objetivo: contribuir ...,Produção agropecuária (42%),Agropecuária (41%),"Agropecuária, projeto (35%)",Pesquisa agropecuária (33%),"Agropecuária, pesquisa (33%)"
3,atenção primária à saúde fortalecer a atenção ...,Atenção primária à saúde (55%),Atenção à saúde (47%),Medicamentos para a atenção primária à saúde (...,Saúde da família (43%),Promoção da saúde (43%)
4,atenção especializada à saúde ampliar o acesso...,Atenção à saúde (45%),Atenção básica à saúde (35%),Atenção básica (saúde) (35%),Serviços de saúde (35%),Atenção primária à saúde (32%)


### Merging dataframes

In [10]:
df_merged = pd.merge(df_propostas, df_classified, on='sentence', how='left')
df_merged.head(5)

,id,category/id,category/name/pt-BR,title/pt-BR,body/pt-BR,supports,followers,comments,published_at,state,url,participatory_space/url,sentence,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5
0,1,30.0,Turismo,Turismo: esse é o Destino,Objetivo: Posicionar o turismo como vetor de d...,1,1,0,2023-05-10 10:03:41 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,turismo: esse é o destino objetivo: posicionar...,Turismo sustentável (37%),Brasileiros (33%),Turismo de saúde (30%),Atividade turística (29%),Turismo social (29%)
1,8,31.0,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,1,0,0,2023-05-10 16:22:51 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agricultura familiar e agroecologia objetivo: ...,Agricultura familiar (41%),Agricultura sustentável (31%),"Agricultura, produção (31%)",Vida familiar (30%),Renda familiar (30%)
2,8,31.0,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,1,0,0,2023-05-10 16:22:51 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agricultura familiar e agroecologia objetivo: ...,Agricultura familiar (41%),Agricultura sustentável (31%),"Agricultura, produção (31%)",Vida familiar (30%),Renda familiar (30%)
3,9,1.0,Agricultura e Pecuária,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,2,0,0,2023-05-10 16:35:47 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agropecuária sustentável objetivo: contribuir ...,Produção agropecuária (42%),Agropecuária (41%),"Agropecuária, projeto (35%)",Pesquisa agropecuária (33%),"Agropecuária, pesquisa (33%)"
4,9,1.0,Agricultura e Pecuária,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,2,0,0,2023-05-10 16:35:47 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agropecuária sustentável objetivo: contribuir ...,Produção agropecuária (42%),Agropecuária (41%),"Agropecuária, projeto (35%)",Pesquisa agropecuária (33%),"Agropecuária, pesquisa (33%)"


### Identifying Participative Processes

In [11]:
conjuv_url = df_merged['participatory_space/url'].tolist()[-1]
conjuv = conjuv_url.split('/')[3] + '|' + conjuv_url.split('/')[4].split('?')[0]  + '|' + conjuv_url.split('=')[-1]
program_url = df_merged['participatory_space/url'].tolist()[0]
program = program_url.split('/')[3] + '|' + program_url.split('/')[4].split('?')[0]  + '|' + program_url.split('=')[-1]
set(df_merged['participatory_space/url'].tolist()), conjuv, program

({'http://brasilparticipativo.presidencia.gov.br/assemblies/confjuv4?assembly_slug=confjuv4',
  'http://brasilparticipativo.presidencia.gov.br/processes/programas?participatory_process_slug=programas'},
 'assemblies|confjuv4|confjuv4',
 'processes|programas|programas')

In [12]:
df_merged['participatory_space'] = df_merged['participatory_space/url'].str.split('/').str[3] + '|' + \
                                  df_merged['participatory_space/url'].str.split('/').str[4].str.split('?').str[0] + '|' + \
                                  df_merged['participatory_space/url'].str.split('/').str[4].str.split('=').str[-1]
df_merged['participatory_space']

0        processes|programas|programas
1        processes|programas|programas
2        processes|programas|programas
3        processes|programas|programas
4        processes|programas|programas
                     ...              
10589     assemblies|confjuv4|confjuv4
10590     assemblies|confjuv4|confjuv4
10591     assemblies|confjuv4|confjuv4
10592     assemblies|confjuv4|confjuv4
10593     assemblies|confjuv4|confjuv4
Name: participatory_space, Length: 10594, dtype: object

### Renaming columns

In [13]:
df_merged_renamed = df_merged[['id', 'state', 'participatory_space', 'category/name/pt-BR', 'supports', 'title/pt-BR', 'body/pt-BR', 'Tópico 1', 'Tópico 2', 'Tópico 3', 'Tópico 4', 'Tópico 5']] \
    .rename(columns={
        'id': 'id_proposta',
        'state': 'status',
        'participatory_space': 'programa',
        'category/name/pt-BR': 'tema',
        'supports': 'votos',
        'title/pt-BR': 'titulo',
        'body/pt-BR': 'proposta'
    })
df_merged_renamed

,id_proposta,status,programa,tema,votos,titulo,proposta,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5
0,1,NaN,processes|programas|programas,Turismo,1,Turismo: esse é o Destino,Objetivo: Posicionar o turismo como vetor de d...,Turismo sustentável (37%),Brasileiros (33%),Turismo de saúde (30%),Atividade turística (29%),Turismo social (29%)
1,8,NaN,processes|programas|programas,Desenvolvimento Agrário e Agricultura Familiar,1,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,Agricultura familiar (41%),Agricultura sustentável (31%),"Agricultura, produção (31%)",Vida familiar (30%),Renda familiar (30%)
2,8,NaN,processes|programas|programas,Desenvolvimento Agrário e Agricultura Familiar,1,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,Agricultura familiar (41%),Agricultura sustentável (31%),"Agricultura, produção (31%)",Vida familiar (30%),Renda familiar (30%)
3,9,NaN,processes|programas|programas,Agricultura e Pecuária,2,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,Produção agropecuária (42%),Agropecuária (41%),"Agropecuária, projeto (35%)",Pesquisa agropecuária (33%),"Agropecuária, pesquisa (33%)"
4,9,NaN,processes|programas|programas,Agricultura e Pecuária,2,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,Produção agropecuária (42%),Agropecuária (41%),"Agropecuária, projeto (35%)",Pesquisa agropecuária (33%),"Agropecuária, pesquisa (33%)"
...,...,...,...,...,...,...,...,...,...,...,...,...
10589,14066,NaN,assemblies|confjuv4|confjuv4,IV - Direito à Diversidade e à Igualdade,0,Bulling Zero nas escolas,O período escolar não somente desenvolve o sab...,Adulto (25%),"Adulto, educação (23%)",Exploração sexual de crianças e adolescentes (...,Ser humano (16%),Probabilidade (15%)
10590,14067,NaN,assemblies|confjuv4|confjuv4,VI - Direito à Cultura,2,Centros Culturais para Jovens,"A cultura é direito fundamental de todos, mas ...",Direito de acesso à cultura (31%),Instrumentos musicais (29%),Direito à cultura (26%),Direitos culturais (23%),Centros de orientação (23%)
10591,14068,NaN,assemblies|confjuv4|confjuv4,VI - Direito à Cultura,1,Passe Cultural para Estudantes,Programa que dá bolsa auxílio para o jovem de ...,Baixa renda (30%),População de baixa renda (28%),Habitação de baixa renda (25%),Jovem (23%),Região metropolitana (20%)
10592,14069,NaN,assemblies|confjuv4|confjuv4,V - Direito à Saúde,0,Políticos e familiares serem obrigados a usare...,"Nosso SUS, a pesar de ser um exemplo sobre mui...",Direitos políticos (19%),Dívida de dinheiro (18%),Dinheiro digital (17%),Sistemas políticos (17%),Relações familiares (17%)


### Saving to file

In [14]:
OUTPUT_FILE = DATA_PATH + 'results_TopicClassification_TFIDF_CosineSimilarity.csv'
df_merged = df_merged.drop_duplicates()
df_merged.to_csv(OUTPUT_FILE, index=False)